# Data Assimilation with Newtonian Nudging 

In [ ]:
import os
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == 'data_assimilation':
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.PDM import PDM
from functions.performance_metrics import NSE, mNSE, FHV
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
import hvplot 
import hvplot.pandas
import itertools
import warnings
from numba import jit
from datetime import datetime

exec_parameter_testing = False
presentation = False

%load_ext autoreload 
%autoreload 2 

In [ ]:
%run "data_assimilation/data_prep.py"

## Necessary data load in 

In [ ]:
#Needed for PDM inputs
preprocess_output_folder = Path('data/Zwalm_data/preprocess_output')
p_zwalm = pd.read_pickle(preprocess_output_folder / 'zwalm_p_thiessen.pkl')
ep_zwalm = pd.read_pickle(preprocess_output_folder / 'zwalm_ep_thiessen.pkl')
param = pd.read_csv("data/Zwalm_PDM_parameters/NM_opt_param.csv")
zwalm_shape = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma_31370.shp')
area_zwalm_new = np.single(zwalm_shape.area[0] / 10**6)
deltat = np.array(1, dtype=np.float32)  # hour
deltat_out = np.array(24, dtype=np.float32)  # daily averaging

#observational C*
ml_obs_op_pad = Path("data/ml_obs_op_data")
Cstar_obs_lin_reg = pd.read_pickle(ml_obs_op_pad/'lin_reg/full_data/y_hat_retimed.pickle')
Cstar_obs_lin_reg_nt = pd.read_pickle(ml_obs_op_pad/'lin_reg/full_data_no_time/y_hat_retimed.pickle')
Cstar_obs_lin_reg_nf = pd.read_pickle(ml_obs_op_pad/'lin_reg/full_data_no_forest/y_hat_retimed.pickle')
# Cstar_obs_ridge_w = pd.read_pickle(ml_obs_op_pad/'ridge/window/y_hat_retimed.pickle')
Cstar_obs_lasso_w = pd.read_pickle(ml_obs_op_pad/'ridge/window/y_hat_retimed.pickle')
Cstar_obs_SVR_lin = pd.read_pickle(ml_obs_op_pad/'SVR/linear/y_hat_retimed.pickle')
Cstar_obs_GPR = pd.read_pickle(ml_obs_op_pad/'GPR/y_hat_retimed.pickle')
#Observational flow for comparison
Q_obs_daily = pd.read_pickle('data/Zwalm_data/pywaterinfo_output/Q_day.pkl')
Q_obs_daily = Q_obs_daily.rename(columns = {'Timestamp':'t'})
Q_obs_daily = Q_obs_daily.set_index('t')
Q_obs_daily.head(2)

Evaluate model performance starting on the first day of the month of first observation. Evaluate based on daily flow!

In [ ]:
first_obs_date = Cstar_obs_lin_reg.index[0]
start_p1 = pd.Timestamp(datetime(year = first_obs_date.year, month = first_obs_date.month, day = 1))
print(f'Start of evaluation: {start_p1}')
end_PDM_calibration = pd.Timestamp(datetime(year = 2019, month = 12, day = 31, hour = 23))
begin_ML_training_only = end_PDM_calibration + np.timedelta64(1,'h')
print(f'End of PDM calibration period: {end_PDM_calibration}')
end_ML_training = pd.Timestamp(datetime(year = 2020, month = 12, day =31))
print(f'End of ML training period: {end_ML_training}')
begin_all_test = end_ML_training + np.timedelta64(1,'D')
end_all_test = Cstar_obs_lin_reg.index[-1]
print(f'Last date used for training {end_all_test}')

## Define general function definition

Define a function to repeatedly compare different Newtonian Nudging parameters and observation operator models

In [ ]:
def DA_OL_comparison(gamma:float, kappa:float, tau:int, Cstar_obs, plot_style = 'dynamic', return_figures = False, combined_figure = False):
    """
    Wrapper for comparing OL and DA (Newtonian Nudging) version of PDM with a certain observation operator model 

    Parameters
    ----------- 
    gamma: float
        the observational uncertainty, as of now fixed for all timestamps (between 0 and 1)
    kappa: float
        The Nudging factor (between 0 and 1)
    tau: int
        The number of hours before and after the time of observation for which to apply DA.
    Cstar_obs: pandas.Series of pandas.DataFrame
        Dataframe/Series with the observed C* (from observation operator model) with time as idex
    plot_style: string
        'dynamic' execued hvplot plotting, 'static' exectued matplotlib plotting, other argument(e.g. None) disable plotting
    return_figures: bool, default = False
        If True, returns fig and axes object of the static plots the order they are displayed
    combined_figure: bool, default = False
        if True, give a combined figure of the last 2 plots

    Returns
    -------
    delta_dict: dictionary
      dictionary containig the differences in NSE between DA and OL for 4 periods:
        - Calibration: from start of observation till the end of PMD calibration
        - ML_training: no PDM calibration, only observation operator model was trained this period
        - Test: both PDM and observation opertor model untrained in this period
        - Full: from start till end of observations
    figs: tuple
        figure objects (only if return_figures = True and plot_style = 'static')
    axes: tuple
        axes objects (only if return_figures = True and plot_style = 'static')

    """
    #Calculate DA and non DA PDM
    pd_zwalm_out_DA = PDM(P=p_zwalm['P_thiessen'].values,
                        EP=ep_zwalm['EP_thiessen'].values,
                        t=p_zwalm['Timestamp'].values,
                        area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                        parameters=param, m=3, DA = True, Cstar_obs = Cstar_obs.values.flatten(),t_obs = Cstar_obs.index.values, gamma = gamma, kappa = kappa,  tau = np.timedelta64(tau,'h'))
    pd_zwalm_out_DA = pd_zwalm_out_DA.set_index('Time')
    pd_zwalm_out = PDM(P=p_zwalm['P_thiessen'].values,
                        EP=ep_zwalm['EP_thiessen'].values,
                        t=p_zwalm['Timestamp'].values,
                        area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                        parameters=param, m=3, DA = False)
    pd_zwalm_out = pd_zwalm_out.set_index('Time')
    Q_out_diff = pd_zwalm_out_DA['qmodm3s'] - pd_zwalm_out['qmodm3s'
                                                           ]
    #Plotting
    diff_Cstar = pd_zwalm_out_DA['Cstar'] - pd_zwalm_out['Cstar']
    if plot_style == 'dynamic':
        display(pd_zwalm_out_DA['Cstar'][start_p1:].hvplot(ylabel='[mm]',
            label = 'C* DA')*pd_zwalm_out['Cstar'][start_p1:].hvplot(label = 'C* OL'))
        display(diff_Cstar[start_p1:].hvplot(ylabel='[mm]', label = r'$\Delta C^* $'))

        display(Q_obs_daily['Value'][start_p1:].hvplot(label = 'Observed')*pd_zwalm_out_DA['qmodm3s'][start_p1:].hvplot(ylabel='[m^3/s]',label = 'DA')*pd_zwalm_out['qmodm3s'][start_p1:].hvplot(label = 'OL',line_dash = 'dotted', frame_width = 800, frame_height = 400))

        display(Q_out_diff[start_p1:].hvplot(title = 'Q_out DA - Q_out OL', ylabel = '[m^3/s]', frame_width = 800))
    
    elif plot_style == 'static':
        fig1, ax1 = plt.subplots()
        pd_zwalm_out_DA['Cstar'][start_p1:].plot(label = 'DA', ylabel = r'$C^*$ [mm]', ax = ax1)
        pd_zwalm_out['Cstar'][start_p1:].plot(label = 'OL', ax = ax1)
        ax1.legend()

        if combined_figure:
            fig2, (ax2, ax3) = plt.subplots(2,1, figsize = (6,7), constrained_layout = True)
        else: 
            fig2, ax2 = plt.subplots(figsize = (8,5))
            fig3, ax3 = plt.subplots(figsize = (8,5))
        Q_obs_daily['Value'][start_p1:].plot(label = 'Observed', ax = ax2)
        pd_zwalm_out_DA['qmodm3s'][start_p1:].plot(ylabel= r'$Q_{out}$ [m$^3$/s]', label = 'DA', ax = ax2)
        pd_zwalm_out['qmodm3s'][start_p1:].plot(label = 'OL',linestyle = 'dotted', ax = ax2)
        ax2.legend()

        Q_out_diff[start_p1:].plot(title = r'$Q_{out}$ DA - $Q_{out}$ OL', ylabel = r'[m$^3$/s]', ax = ax3)

    #Metrics
    def metric_wrapper_DA_OL_comparison(function, metric_name, p_start,p_end):
        metric_OL = function(pd_zwalm_out['qmodm3s'][p_start:p_end],Q_obs_daily['Value'][p_start:p_end])
        metric_DA = function(pd_zwalm_out_DA['qmodm3s'][p_start:p_end],Q_obs_daily['Value'][p_start:p_end])
        print(f'OL {metric_name} from {p_start} till {p_end}: {metric_OL}')
        print(f'DA {metric_name} from {p_start} till {p_end}: {metric_DA}')
        if metric_name == 'FHV':
            delta_metric = np.abs(metric_DA) - np.abs(metric_OL)
        else:
            delta_metric = metric_DA - metric_OL
        print(f'Delta {metric_name}: {delta_metric}')
        return metric_OL, metric_DA, delta_metric
    metric_dict = {'NSE':NSE, 'mNSE':mNSE, 'FHV':FHV}
    delta_dict = {}
    for metric_name in metric_dict.keys():   
        print('\n ------------------')
        print(f'METRIC: {metric_name}')
        print('---------------------')
        metric_OL_cal, metric_DA_cal, delta_cal = metric_wrapper_DA_OL_comparison(metric_dict[metric_name],metric_name,start_p1,end_PDM_calibration)
        print('\n')
        metric_OL_MLt, metric_DA_MLt, delta_MLt =  metric_wrapper_DA_OL_comparison(metric_dict[metric_name],metric_name,begin_ML_training_only,end_ML_training)
        print('\n')
        metric_OL_test, metric_DA_test, delta_test = metric_wrapper_DA_OL_comparison(metric_dict[metric_name],metric_name,begin_all_test,end_all_test)
        print('\n')
        metric_OL_full, metric_DA_full, delta_full = metric_wrapper_DA_OL_comparison(metric_dict[metric_name],metric_name,start_p1,end_all_test)
        # metric_dict = {'OL_cal':metric_OL_cal, 'DA_cal':metric_DA_cal,'OL_ML_training':metric_OL_MLt, 'DA_ML_training':metric_DA_MLt, 'metric_OL_test':metric_OL_test, 'metric_DA_test':metric_DA_test,'metric_OL_full':metric_OL_full, 'metric_DA_full':metric_DA_full}
        delta_dict_temp = {'delta_cal':delta_cal, 'delta_Mlt':delta_MLt,'delta_test':delta_test, 'delta_full':delta_full}
        delta_dict[metric_name] = delta_dict_temp
    if not return_figures:
        return delta_dict
    else:
        if plot_style == 'static':
            if combined_figure:
                figs = (fig1, fig2)
            else:
                figs = (fig1, fig2, fig3)
            axes = (ax1,ax2,ax3)
            return delta_dict, figs, axes
        else:
            raise ValueError("Plot style must be 'static' to allow 'return figures' to be true")
        

Start with $\tau = 5h$ day, $K*\gamma$ = 0.5

# Linear regressinon: full feature set

In [ ]:
kappa = 1
gamma = 0.5
tau = 5
font_size = 13
if presentation:
    plt.rcParams.update({'font.size': font_size})
delta_dict, figs, axes = DA_OL_comparison(gamma, kappa, tau, Cstar_obs_lin_reg, plot_style = 'static', return_figures=True, combined_figure=True)

In [ ]:
pad_pres = Path('Figures/presentation_12_04')
if not os.path.exists(pad_pres):
    os.makedirs(pad_pres)
if presentation:
    if len(figs) == 3: #for if combined figures is F
        fig_diff = figs[2]
        ax_diff = axes[2]
        ax_diff.set_xlabel('Tijd')
        ax_diff.set_title(r'Lineare regressie: $Q_{out}$ DA - $Q_{out}$ OL')
        fig_diff.savefig(pad_pres/'Q_diff_lin_reg.svg',format = 'svg')
        display(fig_diff)
    else:
        fig_combined = figs[1]
        fig_combined.suptitle('Linear regression')
        display(fig_combined)
        fig_combined.savefig(pad_pres/'Q_DA_vs_OL_lin_reg.svg',format = 'svg', transparent = True)


# Linear regression: no time

In [ ]:
DA_OL_comparison(gamma, kappa, tau, Cstar_obs_lin_reg_nt, plot_style = 'dynamic')

# Linear regression: no forest

In [ ]:
DA_OL_comparison(gamma, kappa, tau, Cstar_obs_lin_reg_nf, plot_style = 'dynamic')

# Lasso window regression

In [ ]:
# DA_OL_comparison(gamma, kappa, tau, Cstar_obs_ridge_w['C*'], plot_style = 'dynamic')
DA_OL_comparison(gamma, kappa, tau, Cstar_obs_lasso_w['C*'], plot_style = 'dynamic')

# SVR

In [ ]:
DA_OL_comparison(gamma, kappa, tau, Cstar_obs_SVR_lin['C*'], plot_style = 'dynamic')

# GPR

In [ ]:
dict, figs, axes = DA_OL_comparison(gamma, kappa, tau, Cstar_obs_GPR['C*'], plot_style = 'static', return_figures=True, combined_figure=True)

In [ ]:
if presentation:
    if len(figs) == 3: #for if combined figures is F
        fig_diff = figs[2]
        ax_diff = axes[2]
        ax_diff.set_xlabel('Tijd')
        ax_diff.set_title(r'Gaussiaanse Processen: $Q_{out}$ DA - $Q_{out}$ OL')
        fig_diff.savefig(pad_pres/'Q_diff_gpr.svg',format = 'svg')
        display(fig_diff)
    else:
        fig_combined = figs[1]
        fig_combined.suptitle('Gaussian processes')
        display(fig_combined)
        fig_combined.savefig(pad_pres/'Q_DA_vs_OL_gpr.svg',format = 'svg', transparent = True)

# Comparison of different Newtonian Nudging parameters

Possible parameter combinations:
- $\tau$: 5hours, 0.5, 1 or 1.5 days (not more, since at times 3 days between observations => for more than 1.5 days, code should change to include multiple observations)
- $\gamma K$: 0.1, 0.25, 0.5, 0.75, for which higher means a higher strenght of assimilation

In [ ]:
# pd_Cstar = Cstar_obs_lin_reg.join(
#     [Cstar_obs_lin_reg_nt, Cstar_obs_ridge_w, Cstar_obs_SVR_lin, Cstar_obs_GPR], rsuffix = ['_lin_reg_nt','_ridge_w','_SVR_lin','_GPR']
# )

pd_Cstar = Cstar_obs_lin_reg.join(Cstar_obs_lin_reg_nt['C*'], rsuffix='_lin_reg_nt')
pd_Cstar = pd_Cstar.join(Cstar_obs_ridge_w['C*'], rsuffix='_ridge_w')
pd_Cstar = pd_Cstar.join(Cstar_obs_SVR_lin, rsuffix='_SVR_lin')
pd_Cstar = pd_Cstar.join(Cstar_obs_GPR, rsuffix='_GPR')
pd_Cstar = pd_Cstar.rename(columns = {'C*':'C*_lin_reg'})
display(pd_Cstar)

pad = Path('data/data_assimilation')
if not os.path.exists(pad):
    os.makedirs(pad)

In [ ]:
taus = [1,2,5,int(0.5*24), int(1*24),int(1.5*24)]
gammas = [0.1,0.25,0.5,0.75]
ml_obs_op_models = ['lin_reg','lin_reg_nt','ridge_w','SVR_lin','GPR']
combos = itertools.product(ml_obs_op_models,gammas,taus)
nr_combiations = len(taus)*len(gammas)*len(ml_obs_op_models)
kappa = 1
if exec_parameter_testing:
    for i, combo in enumerate(combos):
        model_name, gamma, tau = combo
        print(f'Combintaion {i} out of {nr_combiations}: tau = {combo[2]} hours, gamma ={combo[1]} and {combo[0]} as observation operator')
        Cstar_temp = pd_Cstar.iloc[:,pd_Cstar.columns.str.endswith(model_name)]
        Cstar_temp = Cstar_temp.dropna() #to deal with window mehtods
        delta_NSE_dict = DA_OL_comparison(kappa, float(gamma), int(tau), Cstar_temp, plot_style = None)
        if i == 0:
            pd_comparison = pd.DataFrame(delta_NSE_dict, index = pd.MultiIndex.from_tuples([combo], names = ['obs_op_model','gamma','tau']))
        else:
            pd_temp = pd.DataFrame(delta_NSE_dict, index = pd.MultiIndex.from_tuples([combo], names = ['obs_op_model','gamma','tau']))
            pd_comparison = pd.concat([pd_comparison, pd_temp])
    pd_comparison.to_pickle(pad/'pd_comparison.pkl')
else:
    pd_comparison = pd.read_pickle(pad/'pd_comparison.pkl')

In [ ]:
max_improv = np.max(pd_comparison.max())
max_deteriation = np.min(pd_comparison.min())
if max_improv < 0:
    warnings.warn('No improvement made!')
limit = np.max([max_improv, np.abs(max_deteriation)])

print(np.max(pd_comparison.max()))
print(np.min(pd_comparison.min()))
pd_comparison.style.background_gradient(cmap = 'coolwarm', vmin =-limit, vmax = limit)#'RdYlGn_r'

In [ ]:
pd_comparison_sort_test = pd_comparison.sort_values('delta_test',ascending = False)
pd_comparison_sort_test.style.background_gradient(cmap = 'coolwarm', vmin =-limit, vmax = limit)

In [ ]:
pd_comparison_sort_full = pd_comparison.sort_values('delta_full',ascending = False)
pd_comparison_sort_full.style.background_gradient(cmap = 'coolwarm', vmin =-limit, vmax = limit)

In [ ]:
-limit

Make scaterplots of model performance based on $\Kappa \gamma$ and $\tau$

In [ ]:
columns_periods = pd_comparison.columns
unique_models = pd_comparison.index.get_level_values('obs_op_model').unique()

figs, axes = plt.subplots(len(unique_models),len(columns_periods), figsize = (12,12), constrained_layout = True)
for i,model in enumerate(unique_models):
    for j,period in enumerate(columns_periods):
        print(str(period) + ', ' + model)
        pd_temp = pd_comparison.loc[(model,), period]
        pd_temp_unstacked = pd_temp.unstack()
        xv, yv = np.meshgrid(pd_temp_unstacked.index.values, 
                             pd_temp_unstacked.columns.values)
        map = axes[i,j].scatter(xv, yv, c = pd_temp_unstacked.values.T,
                                norm = colors.SymLogNorm(vmin = -limit, vmax = limit, linthresh = 1e-4), cmap = 'coolwarm')#, vmin = -limit, vmax = limit, cmap = 'coolwarm')
        axes[i,j].set_xlabel(r'$\gamma$ [-]')
        axes[i,j].set_ylabel(r'$\tau$ [h]')
        if i == 0:
            axes[i,j].set_title(period)
        plt.colorbar(map, ax = axes[i,j])
    plt.setp(axes[i,0], ylabel = model)
#plt.setp(axes[:, 0], ylabel='y axis label')

In [ ]:
unstacked_df = pd_temp.unstack()
display(unstacked_df)
xv, yv = np.meshgrid(unstacked_df.index.values, unstacked_df.columns.values)
plt.scatter(xv, yv, c = unstacked_df.values.T, s= 100)
plt.colorbar()
plt.title('')

## Visualisation of time weighin function

In [ ]:
def tau_weighing(delta_t_abs, tau):
    if delta_t_abs < tau/2:
        W_t = 1
    elif delta_t_abs < tau:
        W_t = (tau - delta_t_abs)/(tau/2)
    else:
        W_t = 0
    return W_t
weights = [tau_weighing(np.abs(delta_t), 12) for delta_t in np.arange(-20,20,1)]
font_size = 13
plt.rcParams.update({'font.size': font_size})
fig, ax = plt.subplots()
ax.plot(np.arange(-20,20), weights)
ax.set_ylabel('$W_t$')
ax.set_xlabel('$t - t^*$ [u]')
pad_pres = Path('Figures/presentation_12_04')
if not os.path.exists(pad_pres):
    os.makedirs(pad_pres)
fig.savefig(pad_pres/'W_t.svg',format = 'svg')
plt.rcParams.update(matplotlib.rcParamsDefault)

# Old experimens only below 

In [ ]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]
tuples = list(zip(*arrays))
print(tuples)
index = pd.MultiIndex.from_tuples(tuples, names = ['first','second'])
print(index)
s = pd.Series(np.random.randn(8),index = index)
s

In [ ]:
np.arange(-30,30)

In [ ]:
def NewtonianNudging(Cstar_min, Cstar_obs, gamma, Kappa, delta_t, tau):
    W_t = tau_weighing(np.abs(delta_t),tau)
    Cstar_plus = Cstar_min + gamma*Kappa*W_t*(Cstar_obs -Cstar_min)
    if Cstar_plus != Cstar_min:
        import pdb; pdb.set_trace()
    return Cstar_plus


In [ ]:
def NN_wrapper(i, t, t_obs, t_a,Cstar,C_star_obs):
    if np.any(np.abs(t[i] - t_obs) < t_a):
        t_assimilated=t_obs[np.abs((t[i] - t_obs)) < t_a]
        print(
            f'{t[i]} should be assimilated since less than {t_a/2} rmeoved from {t_assimilated}')

In [ ]:
C_star_updated = C_star_mod.copy()
for i in range(len(C_star_mod)):
    C_star_min = C_star_mod[i]
    t_mod_i = t_hour[i]
    t_assimilated_index = np.abs(t_mod_i - t_obs).argmin()#t_obs[np.abs((t_hour[i] - t_obs)) < t_a]
    t_assimilated = t_obs[t_assimilated_index]
    delta_t = t_mod_i - t_assimilated
    C_star_updated[i] = NewtonianNudging(C_star_min, Cstar_obs_lin_reg.loc[t_assimilated,:].values[0],0.5,1,delta_t,np.timedelta64(24,'h'))

In [ ]:
t_assimilated 
Cstar_obs_lin_reg.loc[t_assimilated,:].values[0]

In [ ]:
type(Cstar_obs_lin_reg.index.values)

In [ ]:
pd_zwalm_out = PDM(P=p_zwalm['P_thiessen'].values,
                       EP=ep_zwalm['EP_thiessen'].values,
                       t=p_zwalm['Timestamp'].values,
                       area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                       parameters=param, m=3, DA = True, Cstar_obs = Cstar_obs_lin_reg.values.flatten(),t_obs = Cstar_obs_lin_reg.index.values, gamma = 0.5, kappa = 1,  tau = np.timedelta64(12,'h'),)
pd_zwalm_out_DA = pd_zwalm_out.set_index('Time')
pd_zwalm_out = PDM(P=p_zwalm['P_thiessen'].values,
                       EP=ep_zwalm['EP_thiessen'].values,
                       t=p_zwalm['Timestamp'].values,
                       area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                       parameters=param, m=3, DA = False)
pd_zwalm_out = pd_zwalm_out.set_index('Time')

fig, ax = plt.subplots()
pd_zwalm_out_DA['Cstar'].plot(ylabel='[mm]', ax = ax, label = 'C* DA')
pd_zwalm_out['Cstar'].plot(ax = ax, label = 'C* OL')
ax.legend()
pd_zwalm_out.tail()

hvplot.extension('bokeh')
pd_zwalm_out_DA['Cstar'].hvplot(ylabel='[mm]',
    label = 'C* DA')*pd_zwalm_out['Cstar'].hvplot(label = 'C* OL')

Q_obs_daily['Value'].hvplot()

Q_obs_daily['Value'].hvplot(label = 'Observed')*pd_zwalm_out_DA[
    'qmodm3s'].hvplot(ylabel='[m^3/s]',label = 'DA')*pd_zwalm_out['qmodm3s'].hvplot(label = 'OL',line_dash = 'dotted', frame_width = 800, frame_height = 400)

Q_out_diff = pd_zwalm_out_DA['qmodm3s'] - pd_zwalm_out['qmodm3s']
Q_out_diff.hvplot()